In [1]:
import os

In [2]:
%pwd

'c:\\Users\\disom\\Documents\\Github\\MLOps_cancer_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\disom\\Documents\\Github\\MLOps_cancer_classification'

In [5]:
import dagshub
dagshub.init(repo_owner='SA-Duran', repo_name='MLOps_cancer_classification', mlflow=True)

#import mlflow
#with mlflow.start_run():
#  mlflow.log_param('parameter name', 'value')
#  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\disom\Documents\Github\MLOps_cancer_classification\venv\Lib\site-packages\rich\live.py:256: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=8feb15ca-e781-442e-851e-46b47501e525&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ba62150a5c92fd8de0f02340a1a3a4d06e36eb0e037b12bd67bac8adc694510f




Accessing as SA-Duran

Initialized MLflow to track repo "SA-Duran/MLOps_cancer_classification"

Repository SA-Duran/MLOps_cancer_classification initialized!

In [7]:
from src.CancerClassifier.constants.constant import *
from src.CancerClassifier.utils.common import read_yaml, create_directories , save_json
import tensorflow as tf
import mlflow

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    dagshub_repo_owner: str          # ✅ new
    dagshub_repo_name: str           # ✅ new
    params_image_size: list
    params_batch_size: int

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/Chest-CT-Scan-data"),
            all_params=self.params,
            dagshub_repo_owner="SA-Duran",                          # << set yours
            dagshub_repo_name="MLOps_cancer_classification",        # << set yours
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )
        return eval_config

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)  
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        dagshub.init(
            repo_owner=self.config.dagshub_repo_owner,
            repo_name=self.config.dagshub_repo_name,
            mlflow=True,
        )
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": float(self.score[0]), "accuracy": float(self.score[1])})
            # Model registry does not work with file store
            mlflow.keras.log_model(self.model, artifact_path="model", registered_model_name="VGG16Model")



In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e